In [1]:
import pandas as pd
import numpy as np
import io
import re
from tqdm import tqdm

# Printing amount of lines and numbers

In [ ]:
def count_lines(x):
    return sum([1 for i in x.splitlines() if i.strip()])

In [ ]:
def count_numbers(x):
    return len(x)

In [19]:
with open("user_messages_clear_w_dt.json",'r') as fout:
    msg_dt = pd.read_json(fout, compression=None, orient='table')

In [ ]:
print(max(msg_dt['message'].apply(np.vectorize(count_lines))))

In [ ]:
print(max(msg_dt['numbers'].apply(count_numbers)))

# Marking data with key words

In [29]:
def search_by_key_words(x, numbers, key_words):
    for key in key_words:
        key_found = re.search(key, x, re.IGNORECASE)
        if key_found:
            for i, n in enumerate(numbers):
                number_found = re.search(str(n), x[key_found.start():])
                if number_found:
                    if key_found.start() + number_found.end() < len(x):
                        if not x[key_found.start() + number_found.end()].isdigit():
                            if i + 1 < len(numbers):
                                return (len(x[:key_found.start() + number_found.end()].splitlines()), n, numbers[i + 1] if (n < numbers[i + 1]) else n)
                            else:
                                return (len(x[:key_found.start() + number_found.end()].splitlines()), n, n)
    return (None, None, None)

In [27]:
msg_dt['salary_line'] = None
msg_dt['up_fork'] = None
msg_dt['low_fork'] = None

In [21]:
key_words = ['pay', 'fork', 'вилка', 'зп']

In [ ]:
#search_vectorized = lambda x,y: search_by_key_words(x, y, key_words)
#(msg_dt['salary_line'][0], msg_dt['low_fork'][0], msg_dt['up_fork'][0]) = search_vectorized(msg_dt['message'][0], msg_dt['numbers'][0])
#(msg_dt['salary_line'], msg_dt['low_fork'], msg_dt['up_fork']) = np.vectorize(search_vectorized)(msg_dt['message'], msg_dt['numbers'])

In [ ]:
for i in tqdm(msg_dt.index):
    (msg_dt['salary_line'][i], msg_dt['low_fork'][i], msg_dt['up_fork'][i]) = search_by_key_words(msg_dt['message'][i], msg_dt['numbers'][i], key_words)

In [24]:
print(msg_dt[['salary_line', 'low_fork', 'up_fork']])

      salary_line  low_fork  up_fork
0            51.0     100.0    250.0
1            20.0     200.0    250.0
2             7.0      50.0     70.0
3             NaN       NaN      NaN
4             NaN       NaN      NaN
5             NaN       NaN      NaN
6            10.0     200.0    200.0
7            27.0      70.0     90.0
8            44.0      70.0    110.0
9            13.0     150.0    300.0
10            NaN       NaN      NaN
11           17.0     150.0     80.0
12           36.0     140.0    160.0
13           43.0     100.0      0.0
14            9.0     100.0      0.0
15            NaN       NaN      NaN
16           26.0    2500.0   4500.0
17           26.0      80.0    120.0
18           50.0     250.0    300.0
19            4.0     130.0    170.0
20            7.0     480.0    510.0
21            8.0     160.0      0.0
22            4.0     130.0    170.0
23            NaN       NaN      NaN
24            6.0     100.0    140.0
25            NaN       NaN      NaN
2

In [ ]:
with open("user_messages_clear_w_dt.json",'w') as fout:
    msg_dt.to_json(fout, compression=None, orient='table')

# Copying training examples 

In [45]:
with open("user_messages_clear_w_dt.json",'r') as fout:
    msg_dt = pd.read_json(fout, compression=None, orient='table')

In [46]:
msg_dt_true = msg_dt.loc[msg_dt['salary_line'].notnull()]
msg_dt_true.reset_index(inplace=True)
msg_dt_true = msg_dt_true.sample(100).reset_index(drop=True)

In [91]:
def manual_check(msg):
    print()
    lines = msg['message'].splitlines()
    for n, line in enumerate(lines):
        print("{} {}".format(n, line))
    print('Is message a vacancy post? (y/n)')
    c = input()
    if c == 'y':
        print('Input a number of salary line:')
        msg.loc['salary_line'] = int(input())
        print('Input down salary value:')
        msg.loc['low_fork'] = float(input())
        print('Input top salary value:')
        msg.loc['up_fork'] = float(input())
        #return False
    #return True
    return msg

In [107]:
sample_length = 100

msg_dt_false = pd.DataFrame(columns=msg_dt.columns)
msg_dt_unknown = msg_dt.loc[msg_dt['salary_line'].isnull()]
while msg_dt_false.shape[0] < sample_length:
    msg_dt_unknown_sample = msg_dt_unknown.sample(n=sample_length - msg_dt_false.shape[0]).apply(manual_check, axis=1)
    msg_dt_false = msg_dt_false.append(msg_dt_unknown_sample[msg_dt_unknown_sample['salary1_line'].isnull()])
    msg_dt_unknown.drop(msg_dt_unknown_sample.index[msg_dt_unknown_sample['salary_line'].notnull()], inplace=True)
    msg_dt.loc[msg_dt_unknown_sample.index[msg_dt_unknown_sample['salary_line'].notnull()]] = msg_dt_unknown_sample[msg_dt_unknown_sample['salary_line'].notnull()]


0 Всем привет, разыскиваю коллегу
1 Позиция
2 : 
3 Machine Learning Engineer
4  в Must (
5 https://mustapp.com/
6 )
7 Условия
8 : 100 - 150 тыс net, удаленно
9 О нас
10 Must - это небольшая команда творческих, профессиональных и влюбленных в свое дело людей. Наш продукт начался с простой записной книжки фильмов, а сейчас у нас полноценная социальная сеть с фоловингом, друзьями, нотификациями, покупкой билетов и прочими интересными штуками.
11 О задачах
12 Мониторинг и имплементация решений в области персонализации. Конкретные кейсы:
13 - рекомендации фильмов, сериалов, прочего контента
14 - схожесть пользователя и фильма
15 - поиск похожих фильмов и пользователей
16 - анализ рецензий, трендовых тем и персон в киноиндустрии
17 - проведение аб-тестов и оценка значимости результатов
18 Также есть другие ml-задачи в других проектах команды.
19 Отмечу, что в проде уже крутятся решения для персонализации, по этому с нуля продумывать ничего не придется и можно будет относительно быстро увиде

y
Input a number of salary line:
140
Input down salary value:
20
Input top salary value:
25

0 joined #_jobs along with 
1 2 others
2 .
Is message a vacancy post? (y/n)
n

0 https://podolsk.hh.ru/vacancy/19667100
Is message a vacancy post? (y/n)
n

0 Я уже здесь размещал пару-тройку месяцев назад подобный job offer. Опыт оказался положительным, несмотря на баны и занимательные дискуссии. Поэтому опять размещаюсь здесь, предвкушая новый троллинг, баны, и в конечном итоге нахождение нужного человека :))
1 Наш стартап занимается “оцифровкой” мирового рынка предметов питания и созданием на этой основе ряда приложений для потребителей и производителей/поставщиков ( модель “B2B2C”).
2 Наша конечная цель - дать потребителям из любой части света инструмент, позволяющий без усилий и знания диетологии выбирать из всего многообразия предложения именно те продукты, которые наиболее подходят для конкретного человека согласно его образу жизни, физической активности, вкусовым предпочтениям, особеннос

y
Input a number of salary line:
26
Input down salary value:
300
Input top salary value:
300

0 Мы – настоящие монстры обработки данных и адепты маркетинговой аналитики и оптимизации. Последние  
1 7 лет мы работали с Big data в США и теперь мы вернулись с миссией поднять уровень образования и профессионализма на Родине. Вот такой промышленный шпионаж, да!
2 А еще наша мечта – создать настоящее российское Big Data сообщество. 
3 Компания занимается тем, что несет свет бигдаты из Калифорний в немытую Россию
4 (edited)
Is message a vacancy post? (y/n)
n

0 TL;DR — ML researcher в medical imaging стартап (Москва)
1 О нас
2 Мы — Diagnocat, делаем ML-диагноста по медицинским снимкам. Наш пилотный продукт — стоматологическая диагностика по компьютерной томограмме челюсти: на входе DICOM файл с 3D-сканом, на выходе различные визуализации + структурированная диагностика на понятном врачу языке, которую он может сразу прикрепить к истории болезни. Это очень крутая вертикаль в medical imaging (о

n

0 Manifest (MNFST) — это рекламная сеть нового формата. Мы создаем уникальный продукт в сфере influencer marketing, который позволит брендам стать известными большой аудитории без траты времени на согласование рекламных форматов, бюджета, сроков и прочего, а пользователям социальных сетей — монетизировать свой профиль в несколько кликов. Мы запустились в 2018, до этого разработка шла около года, в команде более 20 человек в Москве и Лондоне.
1 Наш сайт: 
2 https://mnfst.com
3 Ищем Python разработчика, который займется сбором и организацией хранения данных.
4 Требования
5 — Опыт программирования на Python/Ruby/Go от 1 года
6 — Опыт работы с  
7 PostgreSQL и нереляционными базами данных MongoDB и Neo4j
8 — Базовое знание HTTP, JS, HTML, DOM
9 — Понимание ООП
10 — Опыт работы по сбору, обработке и хранению данных
11 — Представление о работе современных баз данных
12 Задачи
13 — Работа с RESTful API
14 — Написание многопоточных парсеров
15 — Сборка и минимизация данных
16 — Поставка дан

y
Input a number of salary line:
57
Input down salary value:
120
Input top salary value:
150

0 @sovcharenko
1 : его нет, у меня всего в 2 стартапа были собеседования по теме ds/ml, все остальное в финансах
2 В одной нужен был Lead, который им построит систему скоринга микрокредитов в real time, мне они не понравились + опыта у меня не хватает явно
3 Вторая компания - flocktory, тут уже обсуждали ее
Is message a vacancy post? (y/n)
n

0 нормально чуваки это когда вы говорите что 5%, не нормально - когда вы тратите чужие ресурсы на просчет проектов которых 95% не будет
Is message a vacancy post? (y/n)
n

0 Камрады, а вот в телеграмме вопрос
1 Интересно кому?
2 "Кто мы:
3 Brain Company 
4 brain-company.ru
5  – компания, существующая на базе факультета психологии НИУ ВШЭ, мы занимаемся нейромаркетинговыми исследованиями, то есть такими исследованиями маркетингового материала, которые идут с помощью нейробиологических методов (электроэнцефалограмма, кожно-гальваническая реакция, кардиограм

...

y
Input a number of salary line:
35
Input down salary value:
300
Input top salary value:
300

0 Всем привет! Вакансия не моя, но хорошие ребята из 
1 NAIVE
2  - Neural Artificial Intelligence Video Editor (стартап) ищут:
3 - 
4 специалиста по Deep Learning и Computer Vision
5 , который так же как и NAIVE, любит создавать новые алгоритмы для обработки изображений, аудио и видео, нацелен на развитие и открыт для новых идей и решений. Они разрабатывают приложение по автоматическому монтажу видео для видеографов и простых пользователей, которые хотят получить емкий ролик по итогам отснятого материала с материалов.
6 - 
7 удаленка
8  / 
9 Москва
10  офис 5 мин от м. Бауманская
11 Обязанности:
12 Исследование и разработка DL алгоритмов обработки видео и аудио, интеграция с решения для пользовательского интерфейса и тестирования работы для ускорения монтажа.
13 Требования:
14 - Опыт работы более двух лет;
15 - Знание опыт программирования на Python, C++;
16 - Знание Machine learning, deep lea

/home/rancher/.local/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)



0 я не владею точными цифрами по з/п. По моим ощущениям это 30-300. Т.е. я могу представить людей, кого мы бы взяли на верхний и кого на нижний предел.
Is message a vacancy post? (y/n)
n

0 я не владею точными цифрами по з/п. По моим ощущениям это 30-300. Т.е. я могу представить людей, кого мы бы взяли на верхний и кого на нижний предел.
Is message a vacancy post? (y/n)
n

0 @nyjonx
1  а зарплата от 0 до \inf ?
Is message a vacancy post? (y/n)
n

0 Ищу джуниор дата-аналитика/data-scientistа в команду. Фуллтайм. Можно вообще без опыта работы. 
1 Нужен человек, который заинтересован в дс и мл, cфера интернет-рекламы. Компания международная (
2 weborama.com
3 ), со штабом в Париже, мы же собираем команду в московском офисе. Текущие задачи —  
4 кластеризация пользователей, cкоринговые модели (предсказать покупку), attribution models, анализ поведения пользователей в сети, рекомендательные системы. Из более редких — встречаются задачи по обработке текстов или звонков (которые тоже, впроче

2300

0 joined #_jobs along with 
1 14 others
2 .
Is message a vacancy post? (y/n)
n

0 попросили запостить, все оформлено ок
1 Вакансия Hadoop Engineer в Санкт-Петербурге 
2 от 100,000
3 Компания 
4 https://vk.com/dins_spb
5 Мы ищем талантливых и целеустремленных инженеров для работы в Санкт-Петербургском центре разработок. 
6 Вам предстоит: 
7 • Решать различные задачи по анализу данных используя Hive, Impala и Spark 
8 • Проектировать и создавать ETL в HDFS/Hive и Vertica как с помощью Java/Scala так и с помощью Talend 
9 • Тестировать созданные процессы 
10 Наши ожидания: 
11 • Знание принципов работы Hadoop, MR и HDFS 
12 • Опыт работы с Hive 
13 • У вас есть опыт разработки на Java/Scala 
14 • Умение создавать Oozie Workflow 
15 • Вы уверенный пользователь Linux 
16 • Знание Английского языка и возможность общаться с коллегами из США 
17 Будет плюсом: 
18 • Опыт работы с Apache Spark 
19 • Опыт работы с Talend 
20 • Знание Vertica 
21 • Умение создавать отчеты в Tableau 
22 Мы пр

y
Input a number of salary line:
32
Input down salary value:
7
Input top salary value:
8

0 привет
1 человек ищет работу
2 посмотрите кто кого ищет щас 
3 большой опыт работы руководителем проектов различных направлений, последний год-полтора - "big data" и всякое такое. учит математику и python чтобы и руками уметь работать, пока прошел этак 20% пути по его словам
Is message a vacancy post? (y/n)
n

0 Москва с Питером - наверное как раз на те самые 500-1000 (или скорее процентов 25 от зп). а с другими городами типа Нижнего Новгорода будет уже наверное в два раза
Is message a vacancy post? (y/n)
n

0 @evgenyorlov
1 : 
2 http://www.superjob.ru/research/articles/111737/analitik-big-data/
Is message a vacancy post? (y/n)
n

0 [LaTeX] [opensource] [офтопик]
1 Всем привет! Нам нужно доработать один из пакетов LaTeX: (
2 https://github.com/latex-g7-32/latex-g7-32
3 ).
4 Денег у меня немного, около 10K, зато проектом будет 
5 можно и нужно
6  поделиться с окружающими. В идеале — вообще сделат

**Ключевые слова**
зарплата
salary
оклад
компенсация
з/п
т.р.
гросс
net
yearly

**Валюты**
RUB
UAH
AED
BYN
CAD
MYR

In [109]:
with open("user_messages_clear_w_dt.json",'w') as fout:
    msg_dt.to_json(fout, compression=None, orient='table')

In [48]:
with open("salary_posts.json",'w') as fout:
    msg_dt_true.to_json(fout, compression=None)

In [111]:
with open("non_salary_posts.json",'w') as fout:
    msg_dt_false.to_json(fout, compression=None)

# Lines dataframe

In [77]:
def get_features(msg_df, special_symbols):
        
    cols = ['id',
            'symbol_number', 
            'word_number', 
            'line_number', 
            'number_number', 
            'len_1_lines', 
            'len_2_lines', 
            'special_symbols',
            'isVacancy']
    
    feat_df = pd.Series(0, index=cols)
    
    feat_df['id'] = msg_df['index']
    
    feat_df['symbol_number'] = len(msg_df['message'])
    words = re.split(' |;|,|\*|\n|\t', msg_df['message'])
    lines = msg_df['message'].splitlines()
    
    feat_df['word_number'] = len(words)
    
    feat_df['line_number'] = len(lines)
    
    feat_df['number_number'] = len(msg_df['numbers'])
    
    feat_df['len_1_lines'] = len([x for x in words if len(x) == 1])
    
    feat_df['len_2_lines'] = len([x for x in words if len(x) == 2])
    
    for symbol in special_symbols:
        feat_df['special_symbols'] += msg_df['message'].upper().count(symbol)
        
    if not np.isnan(msg_df['salary_line']):
        feat_df['isVacancy'] = 1
        
    return feat_df

In [65]:
with open("non_salary_posts.json",'r') as fin:
    msg_dt_false = pd.read_json(fin, orient='table')

In [66]:
with open("salary_posts.json",'r') as fin:
    msg_dt_true = pd.read_json(fin, compression=None)

In [51]:
print(msg_dt_true)

    index                                            author  \
0     195  https://opendatascience.slack.com/team/U78BSPA4X   
1    1313  https://opendatascience.slack.com/team/U06J1LG1M   
10     20  https://opendatascience.slack.com/team/UJXTLNABX   
11    665  https://opendatascience.slack.com/team/U4BH7FUCV   
12     18  https://opendatascience.slack.com/team/U2AJE72Q6   
13    963  https://opendatascience.slack.com/team/U1NMUV093   
14    369  https://opendatascience.slack.com/team/U27GFN6P7   
15    812  https://opendatascience.slack.com/team/U0TRC9F1R   
16    522  https://opendatascience.slack.com/team/UA7D8TN9F   
17    445  https://opendatascience.slack.com/team/U1J5L3NDP   
18   2142  https://opendatascience.slack.com/team/U1BMURR5G   
19    737  https://opendatascience.slack.com/team/U7A55LS5A   
2    1340  https://opendatascience.slack.com/team/U14CTBLFJ   
20   1538  https://opendatascience.slack.com/team/U1LHE347N   
21   2283  https://opendatascience.slack.com/team/U3LLR

In [67]:
train_df = pd.concat([msg_dt_false, msg_dt_true], ignore_index=True, sort=False)

In [72]:
cols = ['id',
        'symbol_number', 
        'word_number', 
        'line_number', 
        'number_number', 
        'len_1_lines', 
        'len_2_lines', 
        'special_symbols',
        'isVacancy']
train_feat = pd.DataFrame(columns=cols, index=msg_dt.index)

In [73]:
special_symbols = ["EUR","USD","РУБ","К","ТЫС","€","$","ДОЛЛАР","ЕВРО", "RUB", "UAH", "AED", "BYN", "CAD", "MYR", "Т.Р."]

In [78]:
get_features_vectorized = lambda x: get_features(x, special_symbols)

train_feat = train_df.apply(get_features_vectorized, axis=1)

In [80]:
with open("salary_dataset.json",'w') as fout:
    train_feat.to_json(fout, compression=None, orient='table')

# GradBoost Salary Recognition

In [95]:
import numpy as np
import pandas as pd
import catboost
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

In [82]:
class Model_ctbst:
    def __init__(self, objective='CrossEntropy'):
        '''input:
        objective - loss function
        eval - metric for loggigng'''
        self.objective = objective
        self.model = None
        '''Try to tune it'''
        self.model_params = dict(
            thread_count=8,
            iterations=2000,
            depth=8,
#             bagging_temperature=0.33,
#             learning_rate=0.1,
#             l2_leaf_reg=3,
#             random_strength=0.7,
            loss_function=self.objective
            )
        self.training_params = dict(
            use_best_model=True,
            early_stopping_rounds=50,
            verbose=10
            )
        
    def _set_model_(self):
        self.model = CatBoostClassifier()
        assert self.objective in ['CrossEntropy', 'Logloss']
        self.model.set_params(**self.model_params)
        
    def train(self, X_train, y_train, X_valid, y_valid):
        train_cat_features_indices = np.where(X_train.dtypes != np.float)[0]
        valid_cat_features_indices = np.where(X_valid.dtypes != np.float)[0]
        '''setting pools without weights'''
        ctbst_train_pool = Pool(data=X_train, label=y_train, cat_features=train_cat_features_indices)
        ctbst_val_pool = Pool(data=X_valid, label=y_valid, cat_features=valid_cat_features_indices)
        '''logging'''
        print('Training Model CatBoost')
        print('X_train = %s Y_train = %s' % (X_train.shape, y_train.shape))
        print('X_valid = %s Y_valid = %s' % (X_valid.shape, y_valid.shape))
        print()
        '''training'''
        self._set_model_()
        self.model = self.model.fit(ctbst_train_pool,
                                    eval_set=ctbst_val_pool,
                                    **self.training_params)
        '''feature importances'''
        print('Top features')
        feature_importances = self.model.get_feature_importance(ctbst_train_pool)
        feature_names = X_train.columns
        for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
            print('{}: {}'.format(name, score))
        
    def predict(self, X):
        if self.model is None:
            raise Exception('Train your model before')
        print('Predicting Model CatBoost')
        print('X = %s' % (X.shape,))
        print()
        X_cat_features_indices = np.where(X.dtypes != np.float)[0]
        ctbst_data_pool = Pool(data=X, cat_features=X_cat_features_indices)
        '''predict'''
        prediction = self.model.predict(ctbst_data_pool, prediction_type='Probability')
        '''get pred for 1 class'''
        prediction = pd.Series(prediction[:, 1], index=X.index)
        return prediction


In [286]:
class Trainer:
    def __init__(self, model, train_type, target, features, eval='logloss', n_folds=4, seed=42):
        self.model = model
        self.train_type = train_type
        self.target = target
        self.features = features
        self.eval = eval
        self.n_folds = n_folds
        self.seed = seed
        
    def _generate_folds_(self, data, val_ratio=0.2):
        index = np.asarray(data.index)
        n = index.size
        if self.n_folds == 1:
            random_state = np.random.RandomState(self.seed)
            i_valid = random_state.choice(range(n), size=int(np.floor(n * val_ratio)), replace=False)
            i_train = np.setdiff1d(range(n), i_valid, assume_unique=True)
            folds = [(i_train, i_valid)]
        else:
            fold = KFold(n_splits=self.n_folds, shuffle=True, random_state = self.seed)
            folds = []
            for i_train, i_valid in fold.split(np.arange(n)):
                folds.append((i_train, i_valid))
        return folds
    
    def _get_fold_(self, data, fltr):
        train = data.iloc[fltr[0]].reset_index(drop=True)
        valid = data.iloc[fltr[1]].reset_index(drop=True)
        return train, valid
    
    def _get_error_(self, Y, P):
        assert Y.shape[0] == P.shape[0]
        if self.eval == 'logloss':
            error = log_loss(Y, P)
        elif self.eval == 'roc-auc':
            error = roc_auc_score(Y, P)
        else:
            raise Exception('Error: unknown eval = %s' % (self.eval,))
        return error

    def train(self, data):
        print('Training with %s' % (self.train_type,))
        print()
        if self.train_type == 'validation':
            self._train_with_validation_(data)
        elif self.train_type == 'cross-validation':
            self._train_with_cross_validation_(data)
        else:
            raise Exception('Error: unknown train type = %s' % (self.train_type,))
            
    def _train_basic_(self, train, valid):
        X_train, y_train = train[self.features], train[self.target]
        X_valid, y_valid = valid[self.features], valid[self.target]
        model.train(X_train, y_train, X_valid, y_valid)
        pred_df = model.predict(X_valid)
        error = self._get_error_(y_valid, pred_df)
        print('Error %s: %s' % (self.eval, error))
        print()
        return pred_df, error

    def _train_with_validation_(self, data):
        print('Train with validation...')
        print()
        folds = self._generate_folds_(data)
        train, valid = self._get_fold_(data, folds[0])
        pred_df, error = self._train_basic_(train, valid)
    
    def _train_with_cross_validation_(self, data):
        errors = []
        print('Train with cross-validation...')
        print()
        folds = self._generate_folds_(data)
        print('Cross-validation %d folds' % (self.n_folds,))
        print()
        for i_fold in range(self.n_folds):
            print("Fold = %d / %d" % (i_fold + 1, self.n_folds))
            print()
            train, valid = self._get_fold_(data, folds[i_fold])
            pred_df, error = self._train_basic_(train, valid)
            errors.append(error)
        print('Mean %s error on CV: %s' % (self.eval, np.mean(errors)))
        print()

In [89]:
model = Model_ctbst()

In [210]:
feats = ['symbol_number', 
        'word_number', 
        'line_number', 
        'number_number', 
        'len_1_lines', 
        'len_2_lines', 
        'special_symbols']
target = 'isVacancy'

trainer = Trainer(model=model,  train_type='cross-validation', target=target, 
                  features=feats)

In [115]:
train_test_division = lambda x: (x.iloc[:180], x.iloc[180:])
(train_feat, test_feat) = train_test_division(train_feat.sample(frac=1))

In [116]:
trainer.train(train_feat)

Training with cross-validation

Train with cross-validation...

Cross-validation 4 folds

Fold = 1 / 4

Training Model CatBoost
X_train = (135, 7) Y_train = (135,)
X_valid = (45, 7) Y_valid = (45,)

Learning rate set to 0.047
0:	learn: 0.6576332	test: 0.6634671	best: 0.6634671 (0)	total: 28.9ms	remaining: 57.8s
10:	learn: 0.4300235	test: 0.4781311	best: 0.4781311 (10)	total: 416ms	remaining: 1m 15s
20:	learn: 0.3288142	test: 0.4054306	best: 0.4054306 (20)	total: 944ms	remaining: 1m 28s
30:	learn: 0.2815610	test: 0.3862701	best: 0.3859717 (29)	total: 1.31s	remaining: 1m 22s
40:	learn: 0.2546585	test: 0.3845986	best: 0.3820099 (33)	total: 1.77s	remaining: 1m 24s
50:	learn: 0.2334605	test: 0.3830020	best: 0.3807590 (44)	total: 2.27s	remaining: 1m 26s
60:	learn: 0.2271851	test: 0.3831757	best: 0.3807590 (44)	total: 2.64s	remaining: 1m 23s
70:	learn: 0.2176997	test: 0.3855218	best: 0.3807590 (44)	total: 3.24s	remaining: 1m 28s
80:	learn: 0.2103990	test: 0.3895929	best: 0.3807590 (44)	total:

In [141]:
test_prediction = model.predict(test_feat).to_frame()
test_prediction.columns = ['probability']

print(type(test_feat.isVacancy))

comparison = test_prediction.join(test_feat['isVacancy'])
print(comparison)
roc_auc_score(test_feat['isVacancy'], test_prediction)

Predicting Model CatBoost
X = (20, 9)

<class 'pandas.core.series.Series'>
     probability  isVacancy
129     0.791696          1
2       0.006046          0
138     0.908479          1
66      0.019058          0
76      0.039095          0
22      0.471123          0
108     0.769766          1
96      0.029751          0
100     0.923234          1
19      0.006046          0
13      0.034849          0
8       0.037982          0
171     0.938585          1
182     0.082634          1
118     0.778121          1
151     0.897118          1
125     0.861520          1
14      0.006046          0
50      0.158191          0
97      0.861898          0


0.9292929292929293

In [93]:
model.model.save_model("ClassifierModel",format="cbm", )

# Message recognition by model

In [97]:
#classifier = CatBoostClassifier()
#classifier.load_model("ClassifierModel", format='catboost')

In [99]:
with open("user_messages_clear_w_dt.json",'r') as fin:
    msg_df = pd.read_json(fin, compression=None, orient='table')

In [101]:
msg_df.reset_index(inplace=True)

In [102]:
get_features_vectorized = lambda x: get_features(x, special_symbols)

msg_feat = msg_df.apply(get_features_vectorized, axis=1)

In [109]:
msg_df['probability'] = model.predict(msg_feat)

Predicting Model CatBoost
X = (2431, 9)



In [296]:
with open("user_messages_clear_w_dt_p.json.json",'w') as fout:
    msg_df.to_json(fout, compression=None)

In [272]:
def get_line_features(msg, special_symbols):
        
    cols = ['id',
            'order_number',
            'symbol_number', 
            'word_number',  
            'number_number', 
            'len_1_lines', 
            'len_2_lines', 
            'special_symbols',
            'msg_probability',
            'isSalary']
    
    lines = msg['message'].splitlines()
    
    lines_feat_df = pd.DataFrame(columns=cols)
    
    for i, line in enumerate(lines):
    
        feat_df = pd.Series(0, index=cols)
    
        feat_df['id'] = msg['index']
        
        feat_df['order_number'] = i
    
        feat_df['symbol_number'] = len(line)
        words = re.split(' |;|,|\*|\n|\t', line)
    
        feat_df['word_number'] = len(words)
        
        feat_df['number_number'] = len(re.findall(r'[0-9]+', line))
    
        feat_df['len_1_lines'] = len([x for x in words if len(x) == 1])
    
        feat_df['len_2_lines'] = len([x for x in words if len(x) == 2])
    
        for symbol in special_symbols:
            feat_df['special_symbols'] += line.upper().count(symbol)
        
        if not np.isnan(msg['salary_line']):
            feat_df['msg_probability'] = 1
        
        if i == msg['salary_line']:
            feat_df['isSalary'] = 1
                
        lines_feat_df = lines_feat_df.append(feat_df, ignore_index=True)
        
   # print(lines_feat_df)
    return lines_feat_df

In [203]:
with open("salary_posts.json",'r') as fin:
    train_true = pd.read_json(fin, compression=None)

In [204]:
with open("non_salary_posts.json",'r') as fin:
    train_false = pd.read_json(fin, compression=None, orient='table')

In [205]:
train_feat = pd.concat([train_true, train_false], ignore_index=True, sort=False)

In [268]:
cols = ['id',
            'order_number',
            'symbol_number', 
            'word_number',  
            'number_number', 
            'len_1_lines', 
            'len_2_lines', 
            'special_symbols',
            'msg_probability',
            'isSalary']

lines_feat_ready = pd.DataFrame(columns=cols)

print(lines_feat_ready.columns)

Index(['id', 'order_number', 'symbol_number', 'word_number', 'number_number',
       'len_1_lines', 'len_2_lines', 'special_symbols', 'msg_probability',
       'isSalary'],
      dtype='object')


In [276]:
#get_line_features_vectorized = lambda x: get_line_features(x, special_symbols)
#append_vectorized = lambda x,y: pd.concat([x,y.apply(get_line_features_vectorized, axis=1)], ignore_index=True)

for i in tqdm(train_feat.index):
    lines_feat_cur = get_line_features(train_feat.iloc[i], special_symbols)
    lines_feat_ready = lines_feat_ready.append(lines_feat_cur, ignore_index=True)





  0%|          | 0/200 [00:00<?, ?it/s]



  0%|          | 1/200 [00:00<00:26,  7.62it/s]



  1%|          | 2/200 [00:00<00:24,  8.02it/s]



  2%|▏         | 3/200 [00:00<00:28,  6.84it/s]



  2%|▏         | 4/200 [00:00<00:26,  7.40it/s]



  2%|▎         | 5/200 [00:00<00:34,  5.72it/s]



  3%|▎         | 6/200 [00:00<00:34,  5.64it/s]



  4%|▎         | 7/200 [00:01<00:52,  3.68it/s]



  4%|▍         | 9/200 [00:01<00:40,  4.70it/s]



  6%|▌         | 11/200 [00:01<00:33,  5.62it/s]



  6%|▌         | 12/200 [00:02<00:46,  4.08it/s]



  7%|▋         | 14/200 [00:02<00:50,  3.70it/s]



  8%|▊         | 15/200 [00:03<00:48,  3.85it/s]



  8%|▊         | 16/200 [00:03<00:47,  3.89it/s]



  8%|▊         | 17/200 [00:03<00:39,  4.62it/s]



  9%|▉         | 18/200 [00:03<00:43,  4.19it/s]



 10%|▉         | 19/200 [00:03<00:38,  4.76it/s]



 10%|█         | 20/200 [00:04<00:34,  5.17it/s]



 10%|█         | 21/200 [00:04<00:31,  5.70it/s]



 11%|█         | 22/200 [

In [207]:
line_model = Model_ctbst()

In [287]:
feats = ['order_number',
         'symbol_number', 
         'word_number',  
         'number_number', 
         'len_1_lines', 
         'len_2_lines', 
         'special_symbols',
         'msg_probability']
target = ['isSalary']

line_trainer = Trainer(model=line_model,  train_type='cross-validation', target=target, 
                  features=feats)

In [285]:
print(lines_feat_ready['isSalary'].unique().tolist())

[0, 1]


In [293]:
train_test_division = lambda x: (x.iloc[:4*len(x.index)//5], x.iloc[4*len(x.index)//5:])
(train_line_feat, test_line_feat) = train_test_division(lines_feat_ready.sample(frac=1))

In [294]:
print(train_line_feat.shape)

(7073, 10)


In [295]:
line_trainer.train(train_line_feat)

Training with cross-validation

Train with cross-validation...

Cross-validation 4 folds

Fold = 1 / 4

Training Model CatBoost
X_train = (5304, 8) Y_train = (5304,)
X_valid = (1769, 8) Y_valid = (1769,)

Learning rate set to 0.069103
0:	learn: 0.5801831	test: 0.5798438	best: 0.5798438 (0)	total: 64.7ms	remaining: 2m 9s
10:	learn: 0.1698792	test: 0.1677062	best: 0.1677062 (10)	total: 679ms	remaining: 2m 2s
20:	learn: 0.1056701	test: 0.1006101	best: 0.1006101 (20)	total: 1.28s	remaining: 2m 1s
30:	learn: 0.0789196	test: 0.0702788	best: 0.0702788 (30)	total: 2.14s	remaining: 2m 15s
40:	learn: 0.0705390	test: 0.0604997	best: 0.0604997 (40)	total: 3.04s	remaining: 2m 25s
50:	learn: 0.0682372	test: 0.0581574	best: 0.0581574 (50)	total: 3.74s	remaining: 2m 22s
60:	learn: 0.0664694	test: 0.0564850	best: 0.0564850 (60)	total: 4.72s	remaining: 2m 30s
70:	learn: 0.0651854	test: 0.0552228	best: 0.0552228 (70)	total: 5.56s	remaining: 2m 30s
80:	learn: 0.0635619	test: 0.0542862	best: 0.0542862 (80)

ValueError: Unknown label type: (0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
1739    0
1740    0
1741    0
1742    0
1743    0
1744    0
1745    0
1746    0
1747    0
1748    0
1749    0
1750    0
1751    0
1752    0
1753    0
1754    0
1755    0
1756    0
1757    0
1758    0
1759    0
1760    0
1761    0
1762    0
1763    1
1764    0
1765    0
1766    0
1767    0
1768    0
Name: isSalary, Length: 1769, dtype: object,)